# ODI Data Cleaning & Feature Engineering 

In [6]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [3]:
load_dotenv()

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 11


True

In [7]:
DB_USER = os.getenv("DB_USER")       
DB_PASSWORD = os.getenv("DB_PASSWORD")  
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

In [8]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

## Load Dataset

In [10]:
df = pd.read_sql("Select * from odi_data", con = engine)

In [11]:
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,playing role_striker,major teams_striker,image url_striker,full name_bowler,country_bowler,batting style_bowler,bowling style_bowler,playing role_bowler,major teams_bowler,image url_bowler
0,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,19.2,Australia,Pakistan,MS Wade,GJ Maxwell,...,Wicketkeeper Batter,"Australia, Gujarat Titans, Australia A, Austra...","https://img1.hscicdn.com/image/upload/f_auto,t...",Mohammad Amir,Pakistan,Left hand Bat,Left arm Fast medium,Bowler,"Pakistan, Antigua and Barbuda Falcons, Asia Li...","https://img1.hscicdn.com/image/upload/f_auto,t..."
1,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,19.3,Australia,Pakistan,GJ Maxwell,MS Wade,...,Batting Allrounder,"Australia, Royal Challengers Bangalore, Victor...","https://img1.hscicdn.com/image/upload/f_auto,t...",Mohammad Amir,Pakistan,Left hand Bat,Left arm Fast medium,Bowler,"Pakistan, Antigua and Barbuda Falcons, Asia Li...","https://img1.hscicdn.com/image/upload/f_auto,t..."
2,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,19.4,Australia,Pakistan,MS Wade,GJ Maxwell,...,Wicketkeeper Batter,"Australia, Gujarat Titans, Australia A, Austra...","https://img1.hscicdn.com/image/upload/f_auto,t...",Mohammad Amir,Pakistan,Left hand Bat,Left arm Fast medium,Bowler,"Pakistan, Antigua and Barbuda Falcons, Asia Li...","https://img1.hscicdn.com/image/upload/f_auto,t..."
3,1000887,2016/17,2017-01-13,"Brisbane Cricket Ground, Woolloongabba",1,19.5,Australia,Pakistan,MS Wade,GJ Maxwell,...,Wicketkeeper Batter,"Australia, Gujarat Titans, Australia A, Austra...","https://img1.hscicdn.com/image/upload/f_auto,t...",Mohammad Amir,Pakistan,Left hand Bat,Left arm Fast medium,Bowler,"Pakistan, Antigua and Barbuda Falcons, Asia Li...","https://img1.hscicdn.com/image/upload/f_auto,t..."
4,1000889,2016/17,2017-01-15,Melbourne Cricket Ground,1,11.5,Australia,Pakistan,TM Head,SPD Smith,...,Top order Batter,"Australia, Sunrisers Hyderabad, Washington Fre...","https://img1.hscicdn.com/image/upload/f_auto,t...",Mohammad Junaid Khan,Pakistan,Right hand Bat,Left arm Fast,Bowler,"Pakistan, Abbottabad, Abbottabad Rhinos, Lanca...","https://img1.hscicdn.com/image/upload/f_auto,t..."


In [12]:
print("Raw Shape : ", df.shape)

Raw Shape :  (1385154, 58)


## Drop Duplicates

In [14]:
df = df.drop_duplicates()

## Drop Irrelevant columns (images, urls, etc..)

In [ ]:
df = df.drop(columns = ['image url_striker', 'image url_bowler'], errors = 'ignore')

## Handle Missing Values

In [18]:
df['runs_off_bat'] = df['runs_off_bat'].fillna(0)
df['extras'] = df['extras'].fillna(0)
df['wicket'] = df['wicket'].fillna(0)
df['winner'] = df['winner'].fillna("Unknown")

## Clean Text Fields

In [19]:
df['batting_team'] = df['batting_team'].str.strip()
df['bowling_team'] = df['bowling_team'].str.strip()

In [20]:
print("After Cleaning : ", df.shape)

After Cleaning :  (1385154, 56)


# MATCH-LEVEL FEATURES

In [21]:
match_summary = df.groupby('match_id').agg({
    'runs_off_bat': 'sum',
    'extras': 'sum',
    'wicket': 'sum',
    'winner': 'first',
    'venue': 'first',
    'season': 'first',
    'toss_winner': 'first',
    'toss_decision': 'first',
    'batting_team': lambda x: list(x.unique()),
    'bowling_team': lambda x: list(x.unique())
}).reset_index()

In [22]:
match_summary.rename(columns={
    'runs_off_bat':'total_runs',
    'extras':'total_extras',
    'wicket':'total_wickets'
}, inplace=True)

# OVER-LEVEL FEATURES

In [23]:
df['over'] = df['ball'].astype(int)

In [24]:
over_summary = df.groupby(['match_id','innings','over','batting_team']).agg({
    'runs_off_bat':'sum',
    'extras':'sum',
    'wicket':'sum'
}).reset_index()

## Add cumulative stats per innings

In [25]:
over_summary['cumulative_runs'] = over_summary.groupby(['match_id','innings'])['runs_off_bat'].cumsum()
over_summary['cumulative_wickets'] = over_summary.groupby(['match_id','innings'])['wicket'].cumsum()
over_summary['run_rate'] = over_summary['cumulative_runs'] / over_summary['over']

# MERGE WITH MATCH MEATDATA

In [26]:
over_with_meta = over_summary.merge(
    match_summary[['match_id','winner','venue','season','toss_winner','toss_decision']],
    on='match_id', how='left'
)

# FEATURE ENGINEERING

In [27]:
MAX_OVERS = 50   
over_with_meta['overs_left'] = MAX_OVERS - over_with_meta['over']

## Target Run(per match)

In [29]:
target_runs_per_match = over_with_meta.groupby('match_id')['cumulative_runs'].max().reset_index()
target_runs_per_match.rename(columns={'cumulative_runs':'target_runs'}, inplace=True)

In [30]:
over_with_meta = over_with_meta.merge(target_runs_per_match, on='match_id', how='left')

## Required runs and run rate(only for 2nd innings)

In [31]:
over_with_meta['required_runs'] = np.where(
    over_with_meta['innings']==2,
    over_with_meta['target_runs'] - over_with_meta['cumulative_runs'],
    np.nan
)

In [32]:
over_with_meta['required_run_rate'] = np.where(
    (over_with_meta['innings']==2) & (over_with_meta['overs_left']>0),
    over_with_meta['required_runs'] / over_with_meta['overs_left'],
    np.nan
)

In [33]:
print("Final Over-Level Data Shape :", over_with_meta.shape)

Final Over-Level Data Shape : (225816, 19)


In [34]:
match_summary.to_csv("../data/ODI/odi_match_summary.csv", index=False)
over_with_meta.to_csv("../data/ODI/odi_over_features.csv", index=False)

print("Cleaning & Feature Engineering complete. Files saved.")

Cleaning & Feature Engineering complete. Files saved.
